In [2]:
import pandas as pd

In [14]:
df = pd.read_csv("data/flood_data.csv")
df["Start_Date"] = pd.to_datetime(df["Start_Date"])
df["End_Date"] = pd.to_datetime(df["End_Date"])

df.loc[
    (df["Start_Date"] > pd.Timestamp("2024-09-01"))
    & (df["Start_Date"] < pd.Timestamp("2024-09-31")),
]


DateParseError: day is out of range for month: 2024-09-31